In [ ]:

import os
import pandas as pd


In [51]:
import pandas as pd
import numpy as np

class MatrixLoader:
    def __init__(self, year, agg):
        self.base_url = 'https://github.com/GuilhermeZiegler/input_output/raw/master/input_output/data/iom'
        self.year = year
        self.agg = agg
        self.file_path = f'{self.base_url}/{self.agg}/{self.year}/MIP-BR-CN10-68S-{self.year}.xlsx'
        self.row_indexes = None
        self.column_indexes = None
        self.data_matrix = None
        self.load_data()
        self.aligned_matrix = self.align_matrix()

    def load_data(self):
        xls = pd.ExcelFile(self.file_path)
        for sheet_name in xls.sheet_names:
            if sheet_name == 'Referência':
                continue  # Ignore the 'Referência' sheet
            self.extract_indices_and_matrix(sheet_name, xls)
            break  # Assuming only one relevant sheet needs to be loaded

    def extract_indices_and_matrix(self, sheet_name, xls):
        df = pd.read_excel(xls, sheet_name=sheet_name, header=None)

        # Extract row indexes starting from A5
        row_start_index = 4
        self.row_indexes = df.iloc[row_start_index:, :3].reset_index(drop=True)

        # Extract column indexes starting from D2
        col_start_row, col_end_row = 1, 3
        col_start_col = 3
        self.column_indexes = df.iloc[col_start_row:col_end_row+1, col_start_col:].reset_index(drop=True)

        # Extract data matrix starting from D5
        self.data_matrix = df.iloc[row_start_index:, col_start_col:].reset_index(drop=True)
        self.data_matrix = self.data_matrix.dropna(axis=1, how='all')

    def align_matrix(self):
        # Ensure the data matrix and column indexes have matching shapes
        col_len_data = self.data_matrix.shape[1]
        col_len_index = self.column_indexes.shape[1]

        if col_len_data < col_len_index:
            # Add NaN columns to data matrix
            for _ in range(col_len_index - col_len_data):
                self.data_matrix[f'NaN_{_}'] = np.nan
        elif col_len_data > col_len_index:
            # Truncate the data matrix columns
            self.data_matrix = self.data_matrix.iloc[:, :col_len_index]

        # Create MultiIndex for rows
        row_index = pd.MultiIndex.from_frame(self.row_indexes)

        # Create MultiIndex for columns
        col_index = pd.MultiIndex.from_frame(self.column_indexes.T)

        # Align the data matrix with NaN filling for missing values
        aligned_df = pd.DataFrame(self.data_matrix.values, index=row_index, columns=col_index)
        aligned_df = aligned_df.reindex(index=row_index, columns=col_index)

        return aligned_df

# Exemplo de uso
year = '2010'
agg = '68S'
loader = MatrixLoader(year, agg)

# Acessar os índices de linha
print("Índices de Linha:")
display(loader.row_indexes)

# Acessar os índices de coluna
print("Índices de Coluna:")
display(loader.column_indexes)

# Acessar a matriz de dados
print("Matriz de Dados:")
display(loader.data_matrix)

# Acessar a matriz alinhada com MultiIndex
print("Matriz Alinhada:")
display(loader.aligned_matrix)



Índices de Linha:


,0,1,2
0,0191,"Agricultura, inclusive o apoio à agricultura e...",1.0
1,0192,"Pecuária, inclusive o apoio à pecuária",2.0
2,0280,Produção florestal; pesca e aquicultura,3.0
3,0580,Extração de carvão mineral e de minerais não-m...,4.0
4,0680,"Extração de petróleo e gás, inclusive as ativi...",5.0
...,...,...,...
66,9480,Organizações associativas e outros serviços pe...,67.0
67,9700,Serviços domésticos,68.0
68,NaN,Total\ndo produto,69.0
69,NaN,NaN,NaN


Índices de Coluna:


,3,4,5,6,7,8,9,10,11,12,...,124,125,126,127,128,129,130,131,132,133
0,01911,01912,01913,01914,01915,01916,01917,01918,01919,01921,...,86911,86921,90801,94801,94802,94803,97001,NaN,NaN,NaN
1,"Arroz, trigo e outros cereais",Milho em grão,"Algodão herbáceo, outras fibras da lav. tempor...",Cana-de-açúcar,Soja em grão,Outros produtos e serviços da lavoura temporária,Laranja,Café em grão,Outros produtos da lavoura permanente,"Bovinos e outros animais vivos, prods. animal,...",...,Saúde pública,Saúde privada,"Serviços de artes, cultura, esporte e recreação","Organizações patronais, sindicais e outros ser...","Manutenção de computadores, telefones e objeto...",Serviços pessoais,Serviços domésticos,TOTAL,NaN,Conferência
2,1,2,3,4,5,6,7,8,9,10,...,122,123,124,125,126,127,128,129,NaN,NaN


Matriz de Dados:


,3,4,5,6,7,8,9,10,11,12,...,124,125,126,127,128,129,130,131,133,NaN_0
0,7557,13554,5155,32731,38875,30796,5975,11007,13181,3030,...,0,0,0,0,0,0,0,168861,0,NaN
1,280,1000,6,160,1225,1545,52,409,293,39069,...,0,0,0,0,0,0,0,83448,0,NaN
2,17,30,0,3,24,136,2,18,47,166,...,0,0,0,0,0,0,0,20332,0,NaN
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,14838,0,NaN
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,117330,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0,0,0,0,0,0,0,0,0,0,...,0,0,0,66640,13070,23936,0,104279,0,NaN
67,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,40334,40334,0,NaN
68,8042,14717,5161,32894,40124,32643,6029,11606,13558,42279,...,99864,116247,23204,66640,18483,24019,40334,6599149,0,NaN
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Matriz Alinhada:


0                                                                                    01911  \
1                                                            Arroz, trigo e outros cereais   
2                                                                                    1.0     
0    1                                                  2                                    
0191 Agricultura, inclusive o apoio à agricultura e ... 1.0                           7557   
0192 Pecuária, inclusive o apoio à pecuária             2.0                            280   
0280 Produção florestal; pesca e aquicultura            3.0                             17   
0580 Extração de carvão mineral e de minerais não-me... 4.0                              0   
0680 Extração de petróleo e gás, inclusive as ativid... 5.0                              0   
...                                                                                    ...   
9480 Organizações associativas e outros serviços pes... 67.0                             0   
9700 Serviços domésticos                                68.0                             0   
NaN  Total\ndo produto                                  69.0                          8042   
     NaN                                                NaN                            NaN   
     Conferência                                        NaN                              0   

0                                                                    01912  \
1                                                            Milho em grão   
2                                                                    2.0     
0    1                                                  2                    
0191 Agricultura, inclusive o apoio à agricultura e ... 1.0          13554   
0192 Pecuária, inclusive o apoio à pecuária             2.0           1000   
0280 Produção florestal; pesca e aquicultura            3.0             30   
0580 Extração de carvão mineral e de minerais não-me... 4.0              0   
0680 Extração de petróleo e gás, inclusive as ativid... 5.0              0   
...                                                                    ...   
9480 Organizações associativas e outros serviços pes... 67.0             0   
9700 Serviços domésticos                                68.0             0   
NaN  Total\ndo produto                                  69.0         14717   
     NaN                                                NaN            NaN   
     Conferência                                        NaN              0   

0                                                                                                         01913  \
1                                                            Algodão herbáceo, outras fibras da lav. temporária   
2                                                                                                         3.0     
0    1                                                  2                                                         
0191 Agricultura, inclusive o apoio à agricultura e ... 1.0                                                5155   
0192 Pecuária, inclusive o apoio à pecuária             2.0                                                   6   
0280 Produção florestal; pesca e aquicultura            3.0                                                   0   
0580 Extração de carvão mineral e de minerais não-me... 4.0                                                   0   
0680 Extração de petróleo e gás, inclusive as ativid... 5.0                                                   0   
...                                                                                                         ...   
9480 Organizações associativas e outros serviços pes... 67.0                                                  0   
9700 Serviços domésticos                                68.0                                                  0   
NaN  Total\ndo produto                

In [42]:
loader.column_indexes.shape

(3, 131)

,0,1,2
66,9480,Organizações associativas e outros serviços pe...,67.0
67,9700,Serviços domésticos,68.0
68,NaN,Total\ndo produto,69.0
69,NaN,NaN,NaN
70,NaN,Conferência,NaN


In [46]:
loader.data_matrix

,3,4,5,6,7,8,9,10,11,12,...,123,124,125,126,127,128,129,130,131,133
0,7557,13554,5155,32731,38875,30796,5975,11007,13181,3030,...,0,0,0,0,0,0,0,0,168861,0
1,280,1000,6,160,1225,1545,52,409,293,39069,...,0,0,0,0,0,0,0,0,83448,0
2,17,30,0,3,24,136,2,18,47,166,...,0,0,0,0,0,0,0,0,20332,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,14838,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,117330,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0,0,0,0,0,0,0,0,0,0,...,0,0,0,21873,0,0,0,0,23299,0
66,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,66640,13070,23936,0,104279,0
67,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,40334,40334,0
68,8042,14717,5161,32894,40124,32643,6029,11606,13558,42279,...,58285,99864,116247,23204,66640,18483,24019,40334,6599149,0


In [ ]:
year = '2010'
agg = 'S68'
loader = MatrixLoader(year, agg)

# Access the data matrix for a specific sheet
data_matrix_producao = loader.get_data_matrix('Producao')
print(data_matrix_producao)


In [ ]:
producao_df

In [ ]:
file_path = f"input_output/data/iom/{agg}/{year}"
df = pd.read_excel(file_path)
# excel_reader = ExcelReader(file_path)

In [ ]:
file_path = '..//data//S68//2018//MIP-BR-CN10-68S-2014.xlsx'


In [ ]:
df = pd.read_excel(file_path)
